# § 10  Data Aggregation and Group Operations

## 10.1 GroupBy Mechanics
* Iterating Over Groups
* Selecting a Column or Subset of Columns
* Grouping with Dicts and Series
* Grouping with Functions
* Grouping by Index Levels

## 10.2 Data Aggregation
* Column-Wise and Multiple Function Application
* Returning Aggregated Data Without Row Indexes

## 10.3 Apply: General split-apply-combine
* Suppressing the Group Keys
* Quantile and Bucket Analysis
* Example: Filling Missing Values with Group-Specific Values
* Example: Random Sampling and Permutation
* Example: Group Weighted Average and Correlation
* Example: Group-Wise Linear Regression

## 10.4 Pivot Tables and Cross-Tabulation
* Cross-Tabulations: Crosstab

## 10.5 Conclusion


In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## § 10.1 GroupBy Mechanics

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [3]:
df['data1']

0   -0.204708
1    0.478943
2   -0.519439
3   -0.555730
4    1.965781
Name: data1, dtype: float64

In [4]:
type(df['data1'])

pandas.core.series.Series

In [5]:
df[['data1']]

,data1
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781


In [6]:
type(df[['data1']])

pandas.core.frame.DataFrame

In [7]:
# series groupby anoter series
grouped = df['data1'].groupby(df['key1'])  # 以 series 進行 groupby
grouped

In [8]:
# 對於 SeriesGroupBy object 而言是無法直接看其內容的
for name, group in grouped:
    print('(', name, ')')
    print(group)
    print('---------------------------------------')

( a )
0   -0.204708
1    0.478943
4    1.965781
Name: data1, dtype: float64
---------------------------------------
( b )
2   -0.519439
3   -0.555730
Name: data1, dtype: float64
---------------------------------------


In [9]:
# SeriesGroupBy object 是 split 後的物件，可以作之後的 apply 與 combine
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

In [10]:
type(grouped.mean())

pandas.core.series.Series

In [12]:
grouped.groups

{'a': [0, 1, 4], 'b': [2, 3]}

In [13]:
# 一個 series 也可以被兩個 series 同時 groupby
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

In [14]:
type(means)

pandas.core.series.Series

In [15]:
means.index

MultiIndex([('a', 'one'),
            ('a', 'two'),
            ('b', 'one'),
            ('b', 'two')],
           names=['key1', 'key2'])

In [16]:
# unstack 就是將 multi-index 最底下的往上橫擺
means.unstack()

key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


In [17]:
group_2 = df['data1'].groupby([df['key1'], df['key2']])
for name, group in group_2:
    print(name)
    print(group)
    print('-----------')

('a', 'one')
0   -0.204708
4    1.965781
Name: data1, dtype: float64
-----------
('a', 'two')
1    0.478943
Name: data1, dtype: float64
-----------
('b', 'one')
2   -0.519439
Name: data1, dtype: float64
-----------
('b', 'two')
3   -0.55573
Name: data1, dtype: float64
-----------


In [18]:
group_2.groups

{('a', 'one'): [0, 4], ('a', 'two'): [1], ('b', 'one'): [2], ('b', 'two'): [3]}

In [19]:
# groupby 的 key 不一定是要原資料中已經存在的
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

In [20]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [21]:
# data frame groupby a column name 而不是一個 series
grouped = df.groupby('key1')
grouped

In [22]:
for name, group in grouped:
    print('(', name, ')')
    print(group)
    print('---------------------------------------')

( a )
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
---------------------------------------
( b )
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023
---------------------------------------


In [23]:
df.groupby('key1').mean()          # 以欄位名稱分組

,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


In [24]:
# data frame groupby a list of column names
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

In [25]:
df.groupby(['key1', 'key2']).groups

{('a', 'one'): [0, 4], ('a', 'two'): [1], ('b', 'one'): [2], ('b', 'two'): [3]}

In [26]:
# 要看符合這 groupby key的組合共有幾橫列，可用 size()
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [27]:
type(df.groupby(['key1', 'key2']).size())

pandas.core.series.Series

In [28]:
df.groupby(['key1', 'key2']).size().index

MultiIndex([('a', 'one'),
            ('a', 'two'),
            ('b', 'one'),
            ('b', 'two')],
           names=['key1', 'key2'])

### § 10.1.1 Iterating Over Groups

In [29]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)
    print('----------------------------------')

a
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
----------------------------------
b
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023
----------------------------------


In [30]:
df.groupby('key1').groups

{'a': [0, 1, 4], 'b': [2, 3]}

In [31]:
for name, group in df.groupby('key1'):
    print(name)
    print(type(name))
    print(group)
    print(type(group))
    print('----------------------------------')

a
<class 'str'>
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
<class 'pandas.core.frame.DataFrame'>
----------------------------------
b
<class 'str'>
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023
<class 'pandas.core.frame.DataFrame'>
----------------------------------


In [32]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)
    print('----------------------------------')

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
----------------------------------
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
----------------------------------
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
----------------------------------
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.55573  0.769023
----------------------------------


In [33]:
df.groupby(['key1', 'key2']).groups

{('a', 'one'): [0, 4], ('a', 'two'): [1], ('b', 'one'): [2], ('b', 'two'): [3]}

In [34]:
df.groupby('key1').groups

{'a': [0, 1, 4], 'b': [2, 3]}

In [40]:
# groupby object
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one -0.204708  1.393406
 1    a  two  0.478943  0.092908
 4    a  one  1.965781  1.246435,
 'b':   key1 key2     data1     data2
 2    b  one -0.519439  0.281746
 3    b  two -0.555730  0.769023}

In [37]:
list(df.groupby('key1'))

[('a',
    key1 key2     data1     data2
  0    a  one -0.204708  1.393406
  1    a  two  0.478943  0.092908
  4    a  one  1.965781  1.246435),
 ('b',
    key1 key2     data1     data2
  2    b  one -0.519439  0.281746
  3    b  two -0.555730  0.769023)]

In [38]:
list(df.groupby('key1'))[0]

('a',
   key1 key2     data1     data2
 0    a  one -0.204708  1.393406
 1    a  two  0.478943  0.092908
 4    a  one  1.965781  1.246435)

In [39]:
list(df.groupby('key1'))[0][1]

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
4,a,one,1.965781,1.246435


In [36]:
pieces['a']

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
4,a,one,1.965781,1.246435


In [41]:
type(pieces['a'])

pandas.core.frame.DataFrame

### 對於不同的 columns 作分群

In [42]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [43]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [44]:
type(df.dtypes)

pandas.core.series.Series

In [45]:
# 以橫向的「欄」作區分；groupby 的依據是 dtypes
grouped = df.groupby(df.dtypes, axis=1)

In [46]:
grouped.groups

{float64: ['data1', 'data2'], object: ['key1', 'key2']}

In [47]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### § 10.1.2 Selecting a Column or Subset of Columns

In [48]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [49]:
for keys, group in df.groupby(['key1', 'key2']):
    print(keys)
    print(group)
    print('------------------------------')

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
------------------------------
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
------------------------------
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
------------------------------
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.55573  0.769023
------------------------------


In [50]:
df.groupby(['key1', 'key2']).groups

{('a', 'one'): [0, 4], ('a', 'two'): [1], ('b', 'one'): [2], ('b', 'two'): [3]}

In [52]:
df.groupby(['key1', 'key2']).mean() # ['data2']

data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

In [51]:
# yelding a data frame
df.groupby(['key1', 'key2'])[['data2']].mean() # ['data2']

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

In [53]:
# yelding a series
df.groupby(['key1', 'key2'])['data2'].mean() 

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

In [54]:
df.groupby(['key1', 'key2'])[['data2']]

In [55]:
# DataFrameGroupBy object
df.groupby(['key1', 'key2'])[['data2']].groups

{('a', 'one'): [0, 4], ('a', 'two'): [1], ('b', 'one'): [2], ('b', 'two'): [3]}

In [56]:
# SeriesGroupBy object
df.groupby(['key1', 'key2'])['data2']

In [57]:
# SeriesGroupBy object
df.groupby(['key1', 'key2'])['data2'].groups

{('a', 'one'): [0, 4], ('a', 'two'): [1], ('b', 'one'): [2], ('b', 'two'): [3]}

In [58]:
df['data2']

0    1.393406
1    0.092908
2    0.281746
3    0.769023
4    1.246435
Name: data2, dtype: float64

In [59]:
type(df['data2'])

pandas.core.series.Series

In [60]:
df[['data2']]

,data2
0,1.393406
1,0.092908
2,0.281746
3,0.769023
4,1.246435


In [61]:
type(df[['data2']])

pandas.core.frame.DataFrame

In [62]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

In [63]:
type(df.groupby(['key1', 'key2'])[['data2']].mean()) # 'data2'

pandas.core.frame.DataFrame

In [64]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

In [65]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [66]:
s_grouped.mean()

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

In [67]:
s_grouped = df.groupby(['key1', 'key2'])[['data2']]
s_grouped

In [68]:
s_grouped.mean()

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

### § 10.1.3 Grouping with Dicts and Series

In [69]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [70]:
# 第一種方法使用 dictionary
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [71]:
# 這個也是橫向分群
by_column = people.groupby(mapping, axis=1) # split
by_column.sum() # apply, combine

,blue,red
Joe,0.503905,1.063885
Steve,1.297183,-1.553778
Wes,-1.021228,-1.116829
Jim,0.524712,1.770545
Travis,-4.230992,-2.405455


In [72]:
by_column = people.groupby(mapping, axis=1)

for name, group in by_column:
    print(name)
    print(group)
    print('------------------------------')

blue
               c         d
Joe     0.274992  0.228913
Steve  -0.371843  1.669025
Wes          NaN -1.021228
Jim     0.523772  0.000940
Travis -2.370232 -1.860761
------------------------------
red
               a         b         e
Joe     1.007189 -1.296221  1.352917
Steve   0.886429 -2.001637 -0.438570
Wes    -0.539741       NaN -0.577087
Jim     0.124121  0.302614  1.343810
Travis -0.713544 -0.831154 -0.860757
------------------------------


In [73]:
by_column.groups

{'blue': ['c', 'd'], 'red': ['a', 'b', 'e']}

In [74]:
# 第二種方法使用 series
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [75]:
# 第二種方法使用 series
people.groupby(map_series, axis=1).sum()

,blue,red
Joe,0.503905,1.063885
Steve,1.297183,-1.553778
Wes,-1.021228,-1.116829
Jim,0.524712,1.770545
Travis,-4.230992,-2.405455


In [76]:
people.groupby(map_series, axis=1).groups

{'blue': ['c', 'd'], 'red': ['a', 'b', 'e']}

### § 10.1.4 Grouping with Functions

In [77]:
people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [78]:
# len(index)後的 index 字串長度進行 groupby
people.groupby(len).sum()  

,a,b,c,d,e
3,0.591569,-0.993608,0.798764,-0.791374,2.119639
5,0.886429,-2.001637,-0.371843,1.669025,-0.438570
6,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [80]:
people.groupby(len).sum().index

Int64Index([3, 5, 6], dtype='int64')

In [79]:
people_grouped = people.groupby(len)
for name, group in people_grouped:
    print(name)
    print(group)
    print('------------------------------')


3
            a         b         c         d         e
Joe  1.007189 -1.296221  0.274992  0.228913  1.352917
Wes -0.539741       NaN       NaN -1.021228 -0.577087
Jim  0.124121  0.302614  0.523772  0.000940  1.343810
------------------------------
5
              a         b         c         d        e
Steve  0.886429 -2.001637 -0.371843  1.669025 -0.43857
------------------------------
6
               a         b         c         d         e
Travis -0.713544 -0.831154 -2.370232 -1.860761 -0.860757
------------------------------


In [81]:
people_grouped.groups

{3: ['Joe', 'Wes', 'Jim'], 5: ['Steve'], 6: ['Travis']}

In [82]:
# 額外再加入 index
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()   # 支援傳入函式，搭配 list、series 進行 groupby

a         b         c         d         e
3 one -0.539741 -1.296221  0.274992 -1.021228 -0.577087
  two  0.124121  0.302614  0.523772  0.000940  1.343810
5 one  0.886429 -2.001637 -0.371843  1.669025 -0.438570
6 two -0.713544 -0.831154 -2.370232 -1.860761 -0.860757

In [83]:
grouped_2 = people.groupby([len, key_list])
                           
for keys, group in grouped_2:
    print(keys)
    print(group)
    print('------------------------------')                      

(3, 'one')
            a         b         c         d         e
Joe  1.007189 -1.296221  0.274992  0.228913  1.352917
Wes -0.539741       NaN       NaN -1.021228 -0.577087
------------------------------
(3, 'two')
            a         b         c        d        e
Jim  0.124121  0.302614  0.523772  0.00094  1.34381
------------------------------
(5, 'one')
              a         b         c         d        e
Steve  0.886429 -2.001637 -0.371843  1.669025 -0.43857
------------------------------
(6, 'two')
               a         b         c         d         e
Travis -0.713544 -0.831154 -2.370232 -1.860761 -0.860757
------------------------------


In [84]:
grouped_2.groups

{(3, 'one'): ['Joe', 'Wes'], (3, 'two'): ['Jim'], (5, 'one'): ['Steve'], (6, 'two'): ['Travis']}

### § 10.1.5 Grouping by Index Levels

In [85]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.560145 -1.265934  0.119827 -1.063512  0.332883
1     -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
2      0.286350  0.377984 -0.753887  0.331286  1.349742
3      0.069877  0.246674 -0.011862  1.004812  1.327195

In [86]:
hier_df.columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [87]:
for cty, group in hier_df.groupby(level='cty', axis=1):
    print(cty)
    print(group)
    print('---------------------------------------')

JP
cty          JP          
tenor         1         3
0     -1.063512  0.332883
1     -0.970736 -1.307030
2      0.331286  1.349742
3      1.004812  1.327195
---------------------------------------
US
cty          US                    
tenor         1         3         5
0      0.560145 -1.265934  0.119827
1     -2.359419 -0.199543 -1.541996
2      0.286350  0.377984 -0.753887
3      0.069877  0.246674 -0.011862
---------------------------------------


In [88]:
hier_df.groupby(level='cty', axis=1).groups

{'JP': [('JP', 1), ('JP', 3)], 'US': [('US', 1), ('US', 3), ('US', 5)]}

## § 10.2 Data Aggregation

In [89]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [90]:
grouped = df.groupby('key1')
for key, group in grouped:
    print(key)
    print(group)
    print('----------------------------')

a
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
----------------------------
b
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023
----------------------------


In [91]:
# grouped['data1']

In [92]:
# 取回一個 series
g_3 = grouped['data1']
for key, group in g_3:
    print(key)
    print(group)
    print('----------------------------')

a
0   -0.204708
1    0.478943
4    1.965781
Name: data1, dtype: float64
----------------------------
b
2   -0.519439
3   -0.555730
Name: data1, dtype: float64
----------------------------


In [93]:
grouped[['data1']]

In [94]:
# 取回一個 dataframe
g_4 = grouped[['data1']]
for name, group in g_4:
    print(name)
    print(group)
    print('----------------------------')

a
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
----------------------------
b
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023
----------------------------


In [95]:
grouped['data1'].quantile(0.9)

key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

In [96]:
grouped[['data1']].quantile(0.9)

,data1
key1,
a,1.668413
b,-0.523068


In [98]:
# grouped.groups

In [100]:
grouped.agg?

In [101]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [99]:
grouped.agg(peak_to_peak)       # 自己設定 Aggregation function

C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


In [102]:
grouped.describe()  # 內建多個 Aggregation funciton

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023

In [103]:
g_4

In [106]:
# 雖然有整個 data frame 的資料，但計算時只算 data1
# g_4.agg(peak_to_peak)

In [107]:
# g_3.agg(peak_to_peak)

### § 10.2.1 Column-Wise and Multiple Function Application

In [108]:
tips = pd.read_csv('examples/tips.csv')
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [109]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [111]:
# tips

In [112]:
grouped = tips.groupby(['day', 'smoker'])

In [113]:
grouped_pct = grouped['tip_pct']
for name, group in grouped_pct:
    print(name)
    print(group)
    print('-------------------------------')

('Fri', 'No')
91     0.155625
94     0.142857
99     0.120385
223    0.187735
Name: tip_pct, dtype: float64
-------------------------------
('Fri', 'Yes')
90     0.103555
92     0.173913
93     0.263480
95     0.117750
96     0.146628
97     0.124688
98     0.142789
100    0.220264
101    0.195059
220    0.180921
221    0.259314
222    0.223776
224    0.117735
225    0.153657
226    0.198216
Name: tip_pct, dtype: float64
-------------------------------
('Sat', 'No')
19     0.162228
20     0.227679
21     0.135535
22     0.141408
23     0.192288
         ...   
233    0.136490
235    0.124131
238    0.130338
239    0.203927
242    0.098204
Name: tip_pct, Length: 45, dtype: float64
-------------------------------
('Sat', 'Yes')
56     0.078927
58     0.156584
60     0.158206
61     0.144823
62     0.179673
         ...   
234    0.193175
236    0.079365
237    0.035638
240    0.073584
241    0.088222
Name: tip_pct, Length: 42, dtype: float64
-------------------------------
('Sun', 'No')


In [114]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [115]:
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

```python
def peak_to_peak(arr):
    return arr.max() - arr.min()
```

In [117]:
# grouped_pct.groups

In [118]:
# grouped_pct.quantile?

In [119]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [120]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [121]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [122]:
type(result)

pandas.core.frame.DataFrame

In [123]:
result.columns

MultiIndex([(   'tip_pct', 'count'),
            (   'tip_pct',  'mean'),
            (   'tip_pct',   'max'),
            ('total_bill', 'count'),
            ('total_bill',  'mean'),
            ('total_bill',   'max')],
           )

In [124]:
result.index

MultiIndex([( 'Fri',  'No'),
            ( 'Fri', 'Yes'),
            ( 'Sat',  'No'),
            ( 'Sat', 'Yes'),
            ( 'Sun',  'No'),
            ( 'Sun', 'Yes'),
            ('Thur',  'No'),
            ('Thur', 'Yes')],
           names=['day', 'smoker'])

In [125]:
# 只取出 higher hierarcy 的一部份
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [126]:
# for name, group in grouped:
#     print(name)
#     print(group)
#     print('----------------------------------------------------------------')

In [127]:
# grouped.groups

In [128]:
# 碼代 column 命名
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)] 
grouped['tip_pct', 'total_bill'].agg(ftuples)

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [129]:
# different column with different calculation
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [130]:
# 針對不同的 group 作不同的 apply
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [131]:
# grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
#              'size' : 'sum'})[['tip_pct']].plot()

In [132]:
# grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
#              'size' : 'sum'})['tip_pct'].plot()

### § 10.2.2 Returning Aggregated Data Without Row Indexes

In [133]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [135]:
tips.groupby(['day', 'smoker']).mean() 

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [136]:
type(tips.groupby(['day', 'smoker']).mean())

pandas.core.frame.DataFrame

In [137]:
tips.groupby(['day', 'smoker']).indices

{('Fri', 'No'): array([ 91,  94,  99, 223], dtype=int64),
 ('Fri',
  'Yes'): array([ 90,  92,  93,  95,  96,  97,  98, 100, 101, 220, 221, 222, 224,
        225, 226], dtype=int64),
 ('Sat',
  'No'): array([ 19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
         32,  33,  34,  35,  36,  37,  38,  39,  40,  57,  59,  64,  65,
         66,  68,  70,  71,  74,  75, 104, 108, 110, 111, 212, 227, 228,
        232, 233, 235, 238, 239, 242], dtype=int64),
 ('Sat',
  'Yes'): array([ 56,  58,  60,  61,  62,  63,  67,  69,  72,  73,  76, 102, 103,
        105, 106, 107, 109, 168, 169, 170, 171, 206, 207, 208, 209, 210,
        211, 213, 214, 215, 216, 217, 218, 219, 229, 230, 231, 234, 236,
        237, 240, 241], dtype=int64),
 ('Sun',
  'No'): array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  41,  42,  43,  44,  45,  46,  47,
         48,  49,  50,  51,  52,  53,  54,  55, 112, 113, 114, 115, 116,
        150, 151, 

In [138]:
# 如果 groupby 有兩個 key，會有 hierarchy 的問題，就用 as_index
tips.groupby(['day', 'smoker'], as_index=False).mean() # 不當作 index

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [139]:
tips.groupby(['day', 'smoker'], as_index=False).mean().columns

Index(['day', 'smoker', 'total_bill', 'tip', 'size', 'tip_pct'], dtype='object')

In [140]:
tips.groupby(['day', 'smoker'], as_index=False).mean().index

RangeIndex(start=0, stop=8, step=1)

## § 10.3 Apply: General split-apply-combine

In [141]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:] # 從小到大；-n 得到從大到小的排列
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [142]:
tips_groupby_smoker = tips.groupby('smoker')

In [143]:
for name, group in tips_groupby_smoker:
    print(name)
    print(group)
    print("----------------")

No
     total_bill   tip smoker   day    time  size   tip_pct
0         16.99  1.01     No   Sun  Dinner     2  0.059447
1         10.34  1.66     No   Sun  Dinner     3  0.160542
2         21.01  3.50     No   Sun  Dinner     3  0.166587
3         23.68  3.31     No   Sun  Dinner     2  0.139780
4         24.59  3.61     No   Sun  Dinner     4  0.146808
..          ...   ...    ...   ...     ...   ...       ...
235       10.07  1.25     No   Sat  Dinner     2  0.124131
238       35.83  4.67     No   Sat  Dinner     3  0.130338
239       29.03  5.92     No   Sat  Dinner     3  0.203927
242       17.82  1.75     No   Sat  Dinner     2  0.098204
243       18.78  3.00     No  Thur  Dinner     2  0.159744

[151 rows x 7 columns]
----------------
Yes
     total_bill   tip smoker  day    time  size   tip_pct
56        38.01  3.00    Yes  Sat  Dinner     4  0.078927
58        11.24  1.76    Yes  Sat  Dinner     2  0.156584
60        20.29  3.21    Yes  Sat  Dinner     2  0.158206
61        13

In [144]:
# tips_groupby_smoker.groups

In [145]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [146]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [147]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [148]:
# 照道理應該是直豎欄的 {no, yes} 被轉到橫列上，變成一個很長的橫列；
# 但可能是為了好看，最後的結果是直豎的
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [150]:
pd.DataFrame.describe?

### 其實 groupby 之後結果作 describe()，其實是作以下的動作：
```python
f = lambda x: x.describe()
grouped.apply(f)
```

### § 10.3.1 Suppressing the Group Keys

In [151]:
# group_keys 取消了階層化的 index
tips.groupby('smoker').apply(top) 

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [152]:
tips.groupby('smoker').apply(top).index

MultiIndex([( 'No',  88),
            ( 'No', 185),
            ( 'No',  51),
            ( 'No', 149),
            ( 'No', 232),
            ('Yes', 109),
            ('Yes', 183),
            ('Yes',  67),
            ('Yes', 178),
            ('Yes', 172)],
           names=['smoker', None])

In [153]:
tips_groupby_smoker = tips.groupby('smoker')
for name, group in tips_groupby_smoker:
    print(name)
    print(group)
    print('---------------')

No
     total_bill   tip smoker   day    time  size   tip_pct
0         16.99  1.01     No   Sun  Dinner     2  0.059447
1         10.34  1.66     No   Sun  Dinner     3  0.160542
2         21.01  3.50     No   Sun  Dinner     3  0.166587
3         23.68  3.31     No   Sun  Dinner     2  0.139780
4         24.59  3.61     No   Sun  Dinner     4  0.146808
..          ...   ...    ...   ...     ...   ...       ...
235       10.07  1.25     No   Sat  Dinner     2  0.124131
238       35.83  4.67     No   Sat  Dinner     3  0.130338
239       29.03  5.92     No   Sat  Dinner     3  0.203927
242       17.82  1.75     No   Sat  Dinner     2  0.098204
243       18.78  3.00     No  Thur  Dinner     2  0.159744

[151 rows x 7 columns]
---------------
Yes
     total_bill   tip smoker  day    time  size   tip_pct
56        38.01  3.00    Yes  Sat  Dinner     4  0.078927
58        11.24  1.76    Yes  Sat  Dinner     2  0.156584
60        20.29  3.21    Yes  Sat  Dinner     2  0.158206
61        13.

In [ ]:
# 如果 groupby 只有一個 key，就不會有 hierarchy 的問題，就用 group
# tips_groupby_smoker.groups

In [154]:
tips.groupby('smoker', group_keys=False).apply(top) 

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 10.3.2 Quantile and Bucket Analysis

In [155]:
frame = pd.DataFrame({'data1': np.random.randn(10),
                      'data2': np.random.randn(10)})
frame

,data1,data2
0,-0.919262,-0.955869
1,-1.549106,-0.023493
2,0.022185,-2.304234
3,0.758363,-0.652469
4,-0.660524,-1.218302
5,0.862580,-1.332610
6,-0.010032,1.074623
7,0.050009,0.723642
8,0.670216,0.690002
9,0.852965,1.001543


In [156]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-0.946, -0.343]
1    (-1.552, -0.946]
2      (-0.343, 0.26]
3       (0.26, 0.863]
4    (-0.946, -0.343]
5       (0.26, 0.863]
6      (-0.343, 0.26]
7      (-0.343, 0.26]
8       (0.26, 0.863]
9       (0.26, 0.863]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-1.552, -0.946] < (-0.946, -0.343] < (-0.343, 0.26] < (0.26, 0.863]]

In [157]:
# apply and combine 的動作是將整個 group 的資料作 apply 運算，
# 然後回傳時，會按什麼 format 作合併
# 以下是輸出四個橫列

def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}


grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-1.552, -0.946]",-0.023493,-0.023493,1.0,-0.023493
"(-0.946, -0.343]",-1.218302,-0.955869,2.0,-1.087085
"(-0.343, 0.26]",-2.304234,1.074623,3.0,-0.168657
"(0.26, 0.863]",-1.332610,1.001543,4.0,-0.073383


In [158]:
# apply and combine 的動作是將整個 group 的資料作 apply 運算，
# 然後回傳時，會按什麼 format 作合併
# 以下是輸出四個橫列

def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}


grouped = frame.data2.groupby(quartiles)

In [159]:
grouped.groups

{(-1.552, -0.946]: [1], (-0.946, -0.343]: [0, 4], (-0.343, 0.26]: [2, 6, 7], (0.26, 0.863]: [3, 5, 8, 9]}

In [160]:
grouped.apply(get_stats)

data1                  
(-1.552, -0.946]  min     -0.023493
                  max     -0.023493
                  count    1.000000
                  mean    -0.023493
(-0.946, -0.343]  min     -1.218302
                  max     -0.955869
                  count    2.000000
                  mean    -1.087085
(-0.343, 0.26]    min     -2.304234
                  max      1.074623
                  count    3.000000
                  mean    -0.168657
(0.26, 0.863]     min     -1.332610
                  max      1.001543
                  count    4.000000
                  mean    -0.073383
Name: data2, dtype: float64

In [161]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-1.552, -0.946]",-0.023493,-0.023493,1.0,-0.023493
"(-0.946, -0.343]",-1.218302,-0.955869,2.0,-1.087085
"(-0.343, 0.26]",-2.304234,1.074623,3.0,-0.168657
"(0.26, 0.863]",-1.332610,1.001543,4.0,-0.073383


In [162]:
grouped.apply(get_stats)

data1                  
(-1.552, -0.946]  min     -0.023493
                  max     -0.023493
                  count    1.000000
                  mean    -0.023493
(-0.946, -0.343]  min     -1.218302
                  max     -0.955869
                  count    2.000000
                  mean    -1.087085
(-0.343, 0.26]    min     -2.304234
                  max      1.074623
                  count    3.000000
                  mean    -0.168657
(0.26, 0.863]     min     -1.332610
                  max      1.001543
                  count    4.000000
                  mean    -0.073383
Name: data2, dtype: float64

In [163]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-1.552, -0.946]",-0.023493,-0.023493,1.0,-0.023493
"(-0.946, -0.343]",-1.218302,-0.955869,2.0,-1.087085
"(-0.343, 0.26]",-2.304234,1.074623,3.0,-0.168657
"(0.26, 0.863]",-1.332610,1.001543,4.0,-0.073383


In [164]:
# qcut 分成數量相等的等分, labels = False 表示切分區間以單純數字代表
grouping = pd.qcut(frame.data1, 10, labels=False)
grouping

0    1
1    0
2    4
3    7
4    2
5    9
6    3
7    5
8    6
9    8
Name: data1, dtype: int64

In [165]:
# 合併方便參照
pd.concat([grouping, frame.data1], axis =1)

,data1,data1
0,1,-0.919262
1,0,-1.549106
2,4,0.022185
3,7,0.758363
4,2,-0.660524
5,9,0.862580
6,3,-0.010032
7,5,0.050009
8,6,0.670216
9,8,0.852965


In [166]:
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats)

data1       
0      min     -0.023493
       max     -0.023493
       count    1.000000
       mean    -0.023493
1      min     -0.955869
                  ...   
8      mean     1.001543
9      min     -1.332610
       max     -1.332610
       count    1.000000
       mean    -1.332610
Name: data2, Length: 40, dtype: float64

In [167]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-0.023493,-0.023493,1.0,-0.023493
1,-0.955869,-0.955869,1.0,-0.955869
2,-1.218302,-1.218302,1.0,-1.218302
3,1.074623,1.074623,1.0,1.074623
4,-2.304234,-2.304234,1.0,-2.304234
5,0.723642,0.723642,1.0,0.723642
6,0.690002,0.690002,1.0,0.690002
7,-0.652469,-0.652469,1.0,-0.652469
8,1.001543,1.001543,1.0,1.001543


### § 10.3.3 Example: Filling Missing Values with Group-Specific       Values

In [168]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.622274
2         NaN
3   -0.726213
4         NaN
5    0.051316
dtype: float64

In [169]:
s.fillna(s.mean())

0   -0.432391
1   -0.622274
2   -0.432391
3   -0.726213
4   -0.432391
5    0.051316
dtype: float64

In [170]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -1.157719
New York      0.816707
Vermont       0.433610
Florida       1.010737
Oregon        1.824875
Nevada       -0.997518
California    0.850591
Idaho        -0.131578
dtype: float64

In [171]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan 
data

Ohio         -1.157719
New York      0.816707
Vermont            NaN
Florida       1.010737
Oregon        1.824875
Nevada             NaN
California    0.850591
Idaho              NaN
dtype: float64

In [172]:
group_key = ['East'] * 4 + ['West'] * 4   # 如果需要衣不同分組進行 fillna()，可以先 group，fillna會依分組填值
for name, group in data.groupby(group_key):
    print(name)
    print(group)
    print(type(group))
    print('------------------------------')

East
Ohio       -1.157719
New York    0.816707
Vermont          NaN
Florida     1.010737
dtype: float64
<class 'pandas.core.series.Series'>
------------------------------
West
Oregon        1.824875
Nevada             NaN
California    0.850591
Idaho              NaN
dtype: float64
<class 'pandas.core.series.Series'>
------------------------------


In [173]:
# 一般預設的狀況是 mean 執行時跳過 NaN
data.groupby(group_key).mean()  

East    0.223241
West    1.337733
dtype: float64

In [175]:
# 我們可以彈性的要求，NaN 塞值的時候會根據該 group 的平均值塞！
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -1.157719
New York      0.816707
Vermont       0.223241
Florida       1.010737
Oregon        1.824875
Nevada        1.337733
California    0.850591
Idaho         1.337733
dtype: float64

In [176]:
# 甚至於更好的是分區塊塞不同的預設值
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name]) # 按 groupby 的 name
data.groupby(group_key).apply(fill_func)

Ohio         -1.157719
New York      0.816707
Vermont       0.500000
Florida       1.010737
Oregon        1.824875
Nevada       -1.000000
California    0.850591
Idaho        -1.000000
dtype: float64

### § 10.3.4 Example: Random Sampling and Permutation

In [178]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [179]:
cards

['AH',
 '2H',
 '3H',
 '4H',
 '5H',
 '6H',
 '7H',
 '8H',
 '9H',
 '10H',
 'JH',
 'KH',
 'QH',
 'AS',
 '2S',
 '3S',
 '4S',
 '5S',
 '6S',
 '7S',
 '8S',
 '9S',
 '10S',
 'JS',
 'KS',
 'QS',
 'AC',
 '2C',
 '3C',
 '4C',
 '5C',
 '6C',
 '7C',
 '8C',
 '9C',
 '10C',
 'JC',
 'KC',
 'QC',
 'AD',
 '2D',
 '3D',
 '4D',
 '5D',
 '6D',
 '7D',
 '8D',
 '9D',
 '10D',
 'JD',
 'KD',
 'QD']

In [180]:
card_val

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10]

In [ ]:
# deck

In [181]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [ ]:
# deck.sample?

In [182]:
def draw(deck, n=5):
    return deck.sample(n) # sample by default of no replacement
draw(deck)

5H      5
KH     10
8S      8
9S      9
10C    10
dtype: int64

In [183]:
# 從 index 解析出是什麼花色
get_suit = lambda card: card[-1]       

# 每種花色挑兩張
deck.groupby(get_suit).apply(draw, n=2)

C  KC     10
   8C      8
D  10D    10
   8D      8
H  7H      7
   8H      8
S  JS     10
   QS     10
dtype: int64

In [184]:
deck.groupby_suit = deck.groupby(get_suit)
for name, group in deck.groupby_suit:
    print(name)
    print(group)
    print("-----------")

C
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
dtype: int64
-----------
D
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64
-----------
H
AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64
-----------
S
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
dtype: int64
-----------


In [185]:
# 這個 group_keys=False 最主要是最後呈現結果時，要不要有分類
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

9C     9
6C     6
AD     1
QD    10
8H     8
5H     5
JS    10
3S     3
dtype: int64

In [186]:
# 如果沒有設 group_keys=False，則最後呈現結果時，就會有分類
deck.groupby(get_suit).apply(draw, n=2)

C  JC    10
   2C     2
D  5D     5
   2D     2
H  8H     8
   JH    10
S  6S     6
   JS    10
dtype: int64

### § 10.3.5 Example: Group Weighted Average and Correlation

In [187]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.912414,0.104104
1,a,0.721194,0.712120
2,a,0.910983,0.917448
3,a,-1.020903,0.803585
4,b,-1.413416,0.327113
5,b,1.296608,0.255107
6,b,0.252275,0.495002
7,b,1.127481,0.413611


In [188]:
grouped = df.groupby('category')

# 分完群之後，也是按對應的 weight 作加權平均值運算
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])  # 取加權平均
grouped.apply(get_wavg)

category
a    0.245919
b    0.308312
dtype: float64

In [189]:
# 來看美國大盤與個股的關係
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [190]:
# 計算每天收盤價比前一天的收盤價來得增減多少百分點
rets = close_px.pct_change().dropna()
rets[-3:]

,AAPL,MSFT,XOM,SPX
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974
2011-10-14,0.033225,0.003311,0.022784,0.017380


In [192]:
# 我們要將同一年的歸一個群 (類)
get_year = lambda x: x.year
by_year = rets.groupby(get_year)

In [193]:
for year, group in by_year:
    print(year)
    print(group)
    break

2003
                AAPL      MSFT       XOM       SPX
2003-01-03  0.006757  0.001421  0.000684 -0.000484
2003-01-06  0.000000  0.017975  0.024624  0.022474
2003-01-07 -0.002685  0.019052 -0.033712 -0.006545
2003-01-08 -0.020188 -0.028272 -0.004145 -0.014086
2003-01-09  0.008242  0.029094  0.021159  0.019386
...              ...       ...       ...       ...
2003-12-24  0.030303 -0.003717  0.002080 -0.001807
2003-12-26  0.018627  0.006063  0.005336  0.001691
2003-12-29  0.017324  0.009272  0.013270  0.012401
2003-12-30  0.006623  0.002297  0.002619  0.000144
2003-12-31  0.004699 -0.005500  0.007837  0.002055

[251 rows x 4 columns]


In [194]:
# 個股與大盤的關係，使用 corrwith()
spx_corr = lambda x: x.corrwith(x['SPX'])  # 每個 column 與 SPX 欄位進行相關性比對
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [195]:
# 個股與個股間的連動性
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))  # 個別欄位計算相關性

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [196]:
for year, group in by_year:
    print(year)
    print(group)
    break

2003
                AAPL      MSFT       XOM       SPX
2003-01-03  0.006757  0.001421  0.000684 -0.000484
2003-01-06  0.000000  0.017975  0.024624  0.022474
2003-01-07 -0.002685  0.019052 -0.033712 -0.006545
2003-01-08 -0.020188 -0.028272 -0.004145 -0.014086
2003-01-09  0.008242  0.029094  0.021159  0.019386
...              ...       ...       ...       ...
2003-12-24  0.030303 -0.003717  0.002080 -0.001807
2003-12-26  0.018627  0.006063  0.005336  0.001691
2003-12-29  0.017324  0.009272  0.013270  0.012401
2003-12-30  0.006623  0.002297  0.002619  0.000144
2003-12-31  0.004699 -0.005500  0.007837  0.002055

[251 rows x 4 columns]


In [197]:
# 個股與個股間的連動性
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))  # 個別欄位計算相關性

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### § 10.3.6 Example: Group-Wise Linear Regression

In [198]:
# !conda install statsmodels

### Linear Regression by  ordinary least squares:
$$
\hat y = 
\begin{bmatrix}
x  & 1
\end{bmatrix}
\begin{bmatrix}
\alpha \\ 
\beta
\end{bmatrix}
$$

In [200]:
import statsmodels.api as sm     # 定義回歸函式，最小平方法 ordinary least squares
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [201]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## § 10.4 Pivot Tables and Cross-Tabulation

In [202]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [203]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [204]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [205]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True, aggfunc=sum)             # margins=True 讓表格有 ALL；小計、總計

size                tip_pct                      
smoker          No    Yes  All         No        Yes        All
time   day                                                     
Dinner Fri     6.0   20.0   26   0.418867   1.488126   1.906993
       Sat   115.0  104.0  219   7.112145   6.212055  13.324199
       Sun   167.0   49.0  216   9.126438   3.557756  12.684194
       Thur    2.0    NaN    2   0.159744        NaN   0.159744
Lunch  Fri     3.0   11.0   14   0.187735   1.133620   1.321354
       Thur  110.0   40.0  150   7.053669   2.785676   9.839345
All          403.0  224.0  627  24.058598  15.177232  39.235830

In [206]:
# 這個 aggfunc=len 有一點難猜懂，
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [207]:
# 原來，它就是用來計算次數的 count
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc='count', margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [208]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

### § 10.4.1 Cross-Tabulations: Crosstab

In [209]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_csv(StringIO(data), sep='\s+')

In [210]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [211]:
pd.crosstab(data.Nationality, data.Handedness, margins=True) # 計算分組頻率

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [212]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [213]:
pd.crosstab([tips.time, tips.smoker], tips.day, margins=True)

day            Fri  Sat  Sun  Thur  All
time   smoker                          
Dinner No        3   45   57     1  106
       Yes       9   42   19     0   70
Lunch  No        1    0    0    44   45
       Yes       6    0    0    17   23
All             19   87   76    62  244

In [214]:
# 原來，它就是用來計算次數的 count
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc='count', margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

## § 10.5 Conclusion

# More reference on Groupby with pandas

## 5 tips for data aggregation in pandas
https://towardsdatascience.com/5-tips-for-data-aggregation-in-pandas-7eaa8476dc75

## Groupby, split-apply-combine and pandas
In this tutorial, you'll learn how to use the pandas groupby operation, which draws from the well-known split-apply-combine strategy, on *Netflix movie data*.

https://www.datacamp.com/community/tutorials/pandas-split-apply-combine-groupby?utm_source=adwords_ppc&utm_medium=cpc&utm_campaignid=14989519638&utm_adgroupid=127836677279&utm_device=m&utm_keyword=&utm_matchtype=&utm_network=g&utm_adpostion=&utm_creative=278443377086&utm_targetid=aud-392016246653:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=9040379&gclid=Cj0KCQiA47GNBhDrARIsAKfZ2rAdtuApksVheJY6FdxrzOZonx4VcDhFD_r2h0k-QY-7VDehlj1OJagaAtvNEALw_wcB

## [Pandas教學]善用Pandas套件的Groupby與Aggregate方法提升資料解讀效率
Kaggle網站的星巴克滿意度調查資料集(Starbucks satisfactory survey.csv)

https://www.learncodewithmike.com/2021/04/pandas-groupby-and-aggregate-method.html?m=1

## All Pandas groupby() You Should Know for Grouping Data and Performing Operations

Pandas tips and tricks to help you get started with data analysis

https://towardsdatascience.com/all-pandas-groupby-you-should-know-for-grouping-data-and-performing-operations-2a8ec1327b5